In [1]:
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

In [2]:
_subway = spark.read.format('com.databricks.spark.csv')\
    .options(header='true', inferschema='true').load('data/CARD_SUBWAY_MONTH_202008.csv')

In [3]:
_subway.printSchema()

root
 |-- 사용일자: integer (nullable = true)
 |-- 노선명: string (nullable = true)
 |-- 역명: string (nullable = true)
 |-- 승차총승객수: integer (nullable = true)
 |-- 하차총승객수: integer (nullable = true)
 |-- 등록일자: integer (nullable = true)



In [4]:
_subway.show(10)

+--------+------+-----------------------+------------+------------+--------+
|사용일자|노선명|                   역명|승차총승객수|하차총승객수|등록일자|
+--------+------+-----------------------+------------+------------+--------+
|20200801|중앙선|                   오빈|         249|         239|20200804|
|20200801|중앙선|                   아신|         467|         595|20200804|
|20200801|중앙선|                   국수|         520|         598|20200804|
|20200801| 2호선|                 한양대|        3244|        3022|20200804|
|20200801| 2호선|       왕십리(성동구청)|       11110|        9769|20200804|
|20200801| 2호선|               상왕십리|        8110|        7289|20200804|
|20200801| 2호선|                   신당|        8703|        8731|20200804|
|20200801| 2호선|동대문역사문화공원(DDP)|        8183|        8474|20200804|
|20200801| 2호선|              을지로4가|        5103|        5207|20200804|
|20200801| 2호선|              을지로3가|        7714|        7826|20200804|
+--------+------+-----------------------+------------+------------+--------+
only show

In [31]:
subway=_subway\
    .withColumnRenamed("사용일자", "dataUse")\
    .withColumnRenamed("노선명", "line")\
    .withColumnRenamed("역ID", "stationID")\
    .withColumnRenamed("역명", "station")\
    .withColumnRenamed("승차총승객수", "NInPassengers")\
    .withColumnRenamed("하차총승객수", "NOutPassengers")\
    .withColumnRenamed("등록일자", "dateRegi")

In [6]:

subway.printSchema()

root
 |-- dataUse: integer (nullable = true)
 |-- line: string (nullable = true)
 |-- station: string (nullable = true)
 |-- NInPassengers: integer (nullable = true)
 |-- NOutPassengers: integer (nullable = true)
 |-- dateRegi: integer (nullable = true)



In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType
sum_columns = F.udf(lambda x,y: x+y,IntegerType())

In [8]:
subway=subway.withColumn("total",sum_columns("NInPassengers","NOutPassengers"))

In [9]:
subway.printSchema()

root
 |-- dataUse: integer (nullable = true)
 |-- line: string (nullable = true)
 |-- station: string (nullable = true)
 |-- NInPassengers: integer (nullable = true)
 |-- NOutPassengers: integer (nullable = true)
 |-- dateRegi: integer (nullable = true)
 |-- total: integer (nullable = true)



In [10]:
from pyspark.sql.window import Window
win = Window.partitionBy("line").orderBy(F.col("total").desc()) # line기준 SumInOut 많은것부터 count,.desc()는 내림차순

In [23]:
from pyspark.sql.functions import row_number
from pyspark.sql.functions import rank
subway_rank=subway.withColumn("rank", row_number().over(win))
subway_rank.filter(subway_rank.rank==1).show(26)

+--------+--------------+--------------------+-------------+--------------+--------+------+----------+---------+----+
| dataUse|          line|             station|NInPassengers|NOutPassengers|dateRegi| total|      date|      Day|rank|
+--------+--------------+--------------------+-------------+--------------+--------+------+----------+---------+----+
|20200807|        일산선|                화정|        17591|         17916|20200810| 35507|2020-08-07|   Friday|   1|
|20200814|        장항선|            온양온천|         4528|          4534|20200817|  9062|2020-08-14|   Friday|   1|
|20200807|        경부선|              영등포|        43168|         46982|20200810| 90150|2020-08-07|   Friday|   1|
|20200807|    우이신설선|        북한산보국문|         5767|          5036|20200810| 10803|2020-08-07|   Friday|   1|
|20200812|        분당선|                야탑|        26098|         27415|20200815| 53513|2020-08-12|Wednesday|   1|
|20200812|         7호선|      가산디지털단지|        46117|         46836|20200815| 92953|2020-08-

In [12]:
toDate = F.udf(lambda x: str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])

In [13]:
subway=subway.withColumn("date",toDate(subway.dataUse))

In [14]:
subway=subway.withColumn("Day",F.dayofweek("date")) # 1: 일요일, 2: 월요일... 7:토요일

In [15]:
from pyspark.sql.types import StringType
def dateToStr(x):
    q=""
    if x==1:
        q="Sunday"
    elif x==2:
        q="Monday"
    elif x==3:
        q="Tuesday"
    elif x==4:
        q="Wednesday"
    elif x==5:
        q="Thursday"
    elif x==6:
        q="Friday"
    elif x==7:
        q="Saturday"
    else:
        q="error!"
    return q

In [16]:
dateToStr_udf=F.udf(dateToStr,StringType())

In [17]:

# 숫자로 되어있던 요일을 String으로
subway=subway.withColumn("Day",dateToStr_udf('Day'))

In [21]:
dayCount=subway.groupby('line').pivot('Day').agg({"total":"sum"})

In [22]:
dayCount.show(25)

+--------------+--------+--------+--------+-------+--------+--------+---------+
|          line|  Friday|  Monday|Saturday| Sunday|Thursday| Tuesday|Wednesday|
+--------------+--------+--------+--------+-------+--------+--------+---------+
|        일산선|  738944|  807345|  561959| 392381|  717131|  735412|   733889|
|        장항선|   84187|   91065|   78907|  68037|   72019|   77146|    76949|
|        경부선| 3099082| 3330164| 2573749|1963225| 2877890| 2927549|  2934851|
|    우이신설선|  289007|  326212|  232855| 176868|  284225|  291323|   294205|
|        분당선| 2594521| 2751372| 1857203|1245970| 2464308| 2542687|  2532730|
|         7호선| 4978092| 5484297| 3424446|2354267| 4891009| 4975629|  4964896|
|        수인선|  291246|  319792|  232872| 167942|  278856|  283623|   286097|
|        안산선|  812413|  905466|  676656| 498383|  773801|  801668|   796552|
|         4호선| 3477064| 3719651| 2624746|1775413| 3386256| 3401055|  3406441|
|         1호선| 1618805| 1739011| 1272247| 814265| 1554120| 1573564|